In [40]:
import numpy as np
import tensorflow as tf

# tf.debugging.set_log_device_placement(True) 

# Load dataset

In [44]:
def decode(tensor):
    """
    Converts the 12 uint64s into a 768 float32 tensor.
    """
    masks = tf.convert_to_tensor(2 ** np.arange(64, dtype=np.int64))
    masked = tf.bitwise.bitwise_and(tf.expand_dims(tensor, -1), masks)
    expanded = tf.cast(tf.not_equal(masked, 0), dtype=tf.float32)
    return tf.reshape(expanded, (-1, 3, 768))

# read file
dataset = tf.data.Dataset.list_files('../data/dataset/*.bin')
dataset = tf.data.FixedLengthRecordDataset(dataset, record_bytes=3 * 12 * 8)
dataset = dataset.map(lambda s: tf.reshape(tf.io.decode_raw(s, tf.int64), (3, 12)))
# dataset = dataset.take(2)
dataset = dataset.batch(32)
dataset = dataset.map(decode)

In [45]:
batch = next(iter(dataset.take(1)))
batch.shape

TensorShape([32, 3, 768])

In [46]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(3, 12, figsize=(18, 4))
for kind in range(3):
    for layer in range(12):
        axs[kind][layer].imshow(batch.numpy()[0][kind].reshape((12,8,8))[layer])
plt.show()

# Train